# Translate Chain

In [ ]:
import os

if os.environ.get("TOGETHER_API_KEY") is None:
   os.environ["TOGETHER_API_KEY"] = "paste_your_api_key_here"

In [ ]:
!pip3 install langchain langchain_openai

In [ ]:
import os
from langchain_openai import ChatOpenAI

model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT"
#model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
#model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-Instruct-v0.2"
#model_name = "openchat/openchat-3.5-1210"
#model_name = "Qwen/Qwen1.5-72B-Chat"

print("Provider: TogetherAI")
print("Model: " + model_name)

llm=ChatOpenAI(model_name=model_name,
               openai_api_key=os.environ.get("TOGETHER_API_KEY"),
               openai_api_base="https://api.together.xyz/v1/")

In [ ]:
import textwrap
from langchain_core.prompts import ChatPromptTemplate

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    text = text.lstrip()
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []
    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines
    return '\n'.join(wrapped_lines)

def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(wrap_text(prompt))
    print("")
    print("Response: ")
    print(response)

def print_messages_and_response(messages, response):
    prompt = ChatPromptTemplate(messages=messages)
    print_prompt_and_response(prompt.format(), response)

def get_completion(prompt, temperature=0.0):
    response = llm.invoke(prompt, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

def get_completion_messages(messages, temperature=0.0):
    response=llm.invoke(messages, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

We will analyze a customer review, including translating and summarizing
it. Then try to compose a response

Here are the steps in the analysis:

1.  **Initial Review**: The process starts with a customer review, which
    is provided as a string variable.

2.  **What Language is It?**: The language of the review is identified
    by prompting a language model to determine the language.

3.  **Let’s Translate It**: The review is translated from the original
    language (in this case, German) to English using a language model.

4.  **Let’s Summarize It**: The translated English review is summarized
    into a single sentence by prompting a language model.

5.  **Analyze the sentiment of the original review on a scale of 1 to
    5**: The sentiment of the original review is analyzed on a scale of
    1 (very negative) to 5 (very positive) by prompting a language
    model.

6.  **Categorize the Review Based On the Main Issue**: The main issue or
    concern raised in the translated English review is categorized into
    one of the following categories: Product Quality, Shipping/Delivery,
    Customer Service, Pricing, or Other.

7.  **Suggest Potential Improvements Based On the Review**: At least
    three potential improvements or changes are suggested to address the
    customer’s concerns based on the translated English review.

8.  **Write an individualized email response to the customer in
    German**: A polite and professional email response is drafted in
    German, addressing the customer’s concerns and offering a resolution
    or next steps.

9.  **Translate the email response to English**: The German email
    response is translated to English using a language model.

Lets start with the review text.

### Initial Review

In [ ]:
review = """Ich bin sehr unzufrieden mit dem Produkt, das ich erhalten habe.
Die Qualität ist enttäuschend, und es entspricht nicht den Beschreibungen auf
der Website. Darüber hinaus war die Lieferung viel zu spät, und der
Kundenservice war wenig hilfreich. Ich erwarte eine sofortige Lösung für dieses
Problem."""

### What Language is It?

In [ ]:
what_language_template = """What language is the following review:
{review}
Just state the language name only.
"""

prompt=what_language_template.format(review=review)
what_language_response = get_completion(prompt)
print_prompt_and_response(prompt, what_language_response)

### Lets Translate It

In [ ]:
translate_template ="""Translate the following review to english:
{review}
"""

prompt=translate_template.format(review=review)
translation_response = get_completion(prompt)
print_prompt_and_response(prompt, translation_response)

### Lets Summarize It

In [ ]:
summarize_template ="""Can you summarize the following review in 1 sentence:
{english_review}
"""

prompt=summarize_template.format(english_review=translation_response)
summarize_response = get_completion(prompt)
print_prompt_and_response(prompt, summarize_response)

### Analyze the sentiment on scale of 1(very negative) to 5(very positive)

In [ ]:
sentiment_template = """
Analyze the sentiment of the following review on a scale of 1 to 5, where 1 is
very negative and 5 is very positive:

{review}

Just give the number only.
"""

prompt = sentiment_template.format(review=review)
sentiment_response = get_completion(prompt)
print_prompt_and_response(prompt, sentiment_response)

### Categorize the Review Based On the Main Issue

In [ ]:
categorize_template = """
Categorize the main issue or concern raised in the following review:

{review}

Choose from the following categories: Product Quality, Shipping/Delivery,
Customer Service, Pricing, Other.
Just give the categories only.
"""

prompt = categorize_template.format(review=translation_response)
category_response = get_completion(prompt)
print_prompt_and_response(prompt, category_response)

### Suggest Potential Improvements Based On the Review

In [ ]:
improvement_template = """
Based on the following review, suggest at least three potential improvements or
changes that could address the customer's concerns:

{review}

Give only the three potential improvements as your output.
"""

prompt = improvement_template.format(review=translation_response)
improvements_response = get_completion(prompt)
print_prompt_and_response(prompt, improvements_response)

### Write an individualized email response to the customer in German

In [ ]:
response_template = """
You have received the following review from a customer:

{review}

Please draft a polite and professional email response to the customer in
German, addressing their concerns and offering a resolution or next steps.
"""

prompt = response_template.format(review=review)
german_response = get_completion(prompt)
print_prompt_and_response(prompt, german_response)

### Translate the email response to English

In [ ]:
translate_response_template = """
Translate the following text from German to English:

{german_response}
"""

prompt = translate_response_template.format(german_response=german_response)
english_response = get_completion(prompt)
print_prompt_and_response(prompt, english_response)

### Final Report

In [ ]:
def fix_for_markdown(s):
    return s.replace('\n', '  \n')

from IPython.display import Markdown

final_report = f"""
### Initial Review
{review}

### Language
{what_language_response}

### Translation
{translation_response}

### Summary
{summarize_response}

### Sentiment 1 (very negative) to 5 (very positive)
{sentiment_response}

### Categorized Main Issue
{category_response}

### Potential Improvements
{improvements_response}

### Individualized Email Response
{german_response}

### Translation to English
{english_response}
"""

fixed = final_report.replace('\n', '  \n')

display(Markdown(fixed))